#Data

In [ ]:
#Took a data containing x and y cooridinates of 94000 points 
X = pd.read_csv("coordinates_datasets")

# Pivot Point Matrix(P):
This is a graph matrix of K Nearest Neighbors which act as the pivot points for the rest of the calculation for approximation of the matrix through Nystrom Method .This is created by the brute force method from the points sampled out from the large dataset. 

In [2]:
import random
from sklearn.neighbors import kneighbors_graph

idx = list(range(0, X.shape[0]))
rand_idx = random.sample(idx, 9400)
for i in rand_idx:
  idx.remove(i)

rand_X = []
for i in rand_idx:
  rand_X.append(X[i])

P = kneighbors_graph(rand_X, n_neighbors=5).toarray()

#Reference Point Matrix(R & R'):


In [ ]:
import numpy as np
from scipy.spatial import distance 

k = 9401
R = np.zeros((9400,88714))
R.shape

for i in rand_idx:
  for j in idx:
    if distance.euclidean(X[i], X[j]) < 5:
      R[i][k] = 1
    else:
      R[i][j] = 0
    k += 1

RT = numpy.transpose(R)

#Resultant Matrix(A):
Here we create the resultant matrix which would be used as the affinity matrix in the calculation of graph laplacian for eigen value decomposition.

In [ ]:
P_inv = np.linalg.inv(P)
A = RT* P_inv* R

#Diagnol Matrix(D):


In [ ]:
D = np.diag(A.sum(axis=1))

#Laplacian Matrix(L):

In [ ]:
# calculated the laplacian matrix by taking the difference between the diagnol matrix and the affinity matrix 
L = D-A

#Spectral Clustering:
Here we use the above created Laplacian matrix ..

In [ ]:
# find the eigenvalues and eigenvectors
vals, vecs = np.linalg.eig(L)

# sort
vecs = vecs[:,np.argsort(vals)]
vals = vals[np.argsort(vals)]

# kmeans on first three vectors with nonzero eigenvalues
kmeans = KMeans(n_clusters=4)
kmeans.fit(vecs[:,1:4])
clusters = kmeans.labels_